## Lifetimes

Some coarse initial points:

- Lifetimes enforce that a piece of memory for a given reference is still valid *in its current memory location*. Moves should really play no part in lifetimes since a) when performed for references, they do not change the memory location of what they refer to and b) when performed for non-references (proper name for this?) lifetimes really don't apply.

Lifetime for a given value (references only?) ends when value is moved (see previous: memory location has changed) or dropped. Examples of both: 

In [7]:
/// As an example of moving. After the move, all references, as expected, must be invalidated:
fn i_own_this(a: String) {
    println!("Owned {}", a);
}

fn end_when_moved() {
    let s = String::from("String");
    let ref_s = &s;
    
    println!("{}", ref_s);
    i_own_this(s);
    // println!("{}", ref_s)  // lifetime has ended.
}

fn end_when_dropped() {
    let ref_s: &String;
    {
        // Will get dropped at }
        let s = String::from("String");
        ref_s = &s;
        println!("{}", ref_s);
    }
    // Borrowed value does not life
    // println!("{}", ref_s);
}

### Generic lifetimes

Two references (e.g `&x` and `&y`) given a similar generic lifetime (say the common `'a`) essentially denote that memory for both is still valid for a given area of our code (which is whatever the `'a` is). 

A bit more clearly, a lifetime `'a` is a generic lifetime parameter that can be specialized with a specific scope at compile time, based on the call site. (See [here](https://stackoverflow.com/a/31612025/4952130) for that.) In essense, generic lifetime specifiers, like generic types, allow us to re-use code containing references for many different instantiations of lifetimes.

In [ ]:
/// The oh-so familiar example:
fn generic_lifetime<'a>(ref1: &'a String, ref2: &'a String) -> &'a String {
    if ref1.len() > ref2.len() {
        return ref1;
    }
    ref2
}

/// Different instantiations of generic lifetime:
fn run(){
    let (value1, value2) = (String::from("value_1"), String::from("value_2"));
    // Lifetime for these references starts now.
    let (ref1, ref2) = (&value1, &value2);
    // First instantiation. Lifetime 'a will be instantiated with lifetime set for
    // ref1, ref2. result (since it refers to one of the two strings), gets the same
    // lifetime.
    let result = generic_lifetime(ref1, ref2);
    {
        // New scope. At end of scope, values get dropped => lifetimes end.
        let (svalue1, svalue2) = (String::from("svalue_1"), String::from("svalue_2"));
        // Lifetime for references starts now.
        let (sref1, sref2) = (&svalue1, &svalue2);

        // Can re-use same function since lifetime is generic.
        // Second instantiation. Lifetime 'a will be instantiated with lifetime set for
        // sref1, sref2. sresult (since it refers to one of the two strings), gets the same
        // lifetime.
        let sresult = generic_lifetime(sref1, sref2);
        
        // lifetime for sref1, sref2, sresult, ends here.
        println!("Result: {}", sresult);    
    }
    
    // lifetime for ref1, ref2, result, ends here.
    println!("Result: {}", result);
}

run()

#### E0106

An example of missing lifetime specifier:

In [13]:
/// No references passed in so ref returned would have to be returned from in function.
/// Also knows that references from inside the function would (probably? definitely?) that all
/// variables created inside the function will be cleaned when the function is finished (end of scope)
/// Applying 'static to it does not help (rust understands that we still return ref to local.)
fn missing_1() -> &i32 {
    let a = 1;
    &a
}

#### E0621

Mismatch in lifetime specifiers.

In [2]:
/// Conditionally return ref.
/// One possible solution: subtyping for lifetimes, i.e place a bound on
/// 'b to tell the compiler to *only* accept lifetimes >= 'b. 
fn condition_ref_return<'a, 'b>(a: &'a i32, b: &'b i32) -> &'a i32 {
    if a > b {
        a
    } else {
        b
    }
}

### Related Errors:

 - [E0106](https://doc.rust-lang.org/error-index.html#E0106)
 - [E0495](https://doc.rust-lang.org/error-index.html#E0495)
 - [E0515](https://doc.rust-lang.org/error-index.html#E0515)
 - [E0621](https://doc.rust-lang.org/error-index.html#E0621)

### Related Videos, Blogs

Videos:

- [YouTube tutorial 1](https://www.youtube.com/watch?v=rAl-9HwD858)
- [YouTube tutorial 2](https://www.youtube.com/watch?v=MSi3E5Z8oRw&list=WL&index=17)

Blogs, Github, StackOverflow:

- [Pretzelhammer - Common Rust lifetime misconceptions](https://github.com/pretzelhammer/rust-blog/blob/master/posts/common-rust-lifetime-misconceptions.md)
- [SO 1](https://stackoverflow.com/questions/25730586/how-can-i-create-my-own-data-structure-with-an-iterator-that-returns-mutable-ref)
- [Generally StackOverflow [rust][lifetime] questions](https://stackoverflow.com/questions/tagged/lifetime+rust)
- Iterators yielding mutable references [part one](http://smallcultfollowing.com/babysteps/blog/2013/10/24/iterators-yielding-mutable-references/) and [two](http://smallcultfollowing.com/babysteps/blog/2013/10/24/iterators-yielding-mutable-references-take-2/)
- [Rust borrow and lifetimes (2015)](http://arthurtw.github.io/2014/11/30/rust-borrow-lifetimes.html).

### More examples:

1. Iterate over a collection of `T` and give out mutable references to items. Basically an `iter_mut`.

(Also an example of E0495)
Aside: There's an interesting question here. Why does the `&mut` case complain when the `&` case compiles just fine? Think of this and google it.

In [20]:
// Since struct is holding a ref, we *need* to specify how long that will live.
// Read as: IterWrapper lives *as long as* reference it contains lives. This *does* mean that
// `slice` can live longer, though. IterWrapper *can't*.
struct IterWrapper<'a, T> {
    slice: &'a [T],
}

// Associated item "Item" must be tied to our IterWrapper. In essense, we still need
// to explicitly state that item returned from next (which is a reference into our slice) 
// lives as long as the `slice` we we've wrapped.
impl<'a, T> Iterator for IterWrapper<'a, T>{
    type Item = &'a T;
    
    fn next(&mut self) -> Option<Self::Item> {
        if self.slice.is_empty() {
            return None;
        }
        // Grab element, trim slice.
        let element = self.slice.get(0);
        self.slice = &self.slice[1..];
        element
    }
}

fn test_iterwrapper(){
    let v = vec![1, 2, 3, 4];
    let iw = IterWrapper { slice: &v[..] };
    
    for value in iw {
         println!("{}", value);
    }
}

test_iterwrapper()

The struct for which we will implement `Iterator`:

In [2]:
struct MutIterWrapper<'a, T> {
    slice: &'a mut [T],
}

First stab at implementing `Iterator` for MutIterWrapper:

In [23]:
impl<'a, T> Iterator for MutIterWrapper<'a, T>{
    type Item = &'a mut T;
    
    fn next(&mut self) -> Option<Self::Item> {
        let result = self.slice.get_mut(0);
        self.slice = &mut self.slice[1..];
        result
    }
}

In [5]:
use std::mem;

impl<'a, T> Iterator for MutIterWrapper<'a, T>{
    type Item = &'a mut T;
    
    fn next(&mut self) -> Option<Self::Item> {
        // Replace self.slice with something else (empty slice)
        // while we work on its copy.
        let slice = mem::replace(&mut self.slice, &mut []);
        // then we replace self.slice with slice from which
        // we took our result
        let (result, rest) = slice.split_first_mut()?;
        self.slice = rest;
        Some(result)
    }
}

In [6]:
fn test_mutiterwrapper(){
    let mut v = vec![1, 2, 3, 4];
    {
        let iw = MutIterWrapper { slice: &mut v[..] };
    
        for value in iw {
             *value = *value + 1;
        }
    }
    for value in v {
        println!("{}", value);
    }
}

test_mutiterwrapper()

2
3
4
5


()

### todo?

- static lifetimes
- lifetime ellision